In [1]:
 !pip install gradio google-generativeai pillow

In [2]:
!pip install transformers>=4.52.4 torchaudio peft soundfile


In [3]:
!pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 20.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=493ebbe6b234ad283f639996a2032bdacfbf2990afdf328b5dead64836f30b45
  Stored in directory: /root/.cache/pip/wheels/c1/67/88/e844b5b022812e15a52e4eaa38a1e709e99f06f6639d7e3ba7
Successfully built langdetect


In [4]:
!pip install gtts

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 5.9 MB/s eta 0:00:00
  Attempting uninstall: click
    Found existing installation: click 8.3.0
    Uninstalling click-8.3.0:
      Successfully uninstalled click-8.3.0


In [5]:
!pip install PyPDF2 gradio google-generativeai gtts langdetect transformers>=4.52.4 torchaudio peft soundfile huggingface-hub

In [ ]:
import gradio as gr
import google.generativeai as genai
from gtts import gTTS
import io
import base64
from langdetect import detect
import langdetect
from langdetect.lang_detect_exception import LangDetectException
import warnings
warnings.filterwarnings("ignore")

# IBM Granite model imports
import torch
import torchaudio
from transformers import AutoProcessor, AutoModelForSpeechSeq2Seq
from huggingface_hub import hf_hub_download
import soundfile as sf
import tempfile
import os
import PyPDF2

class MultilingualTTSWithGranite:
    def __init__(self):
        self.gemini_model = None
        self.granite_model = None
        self.granite_processor = None
        self.granite_tokenizer = None
        self.device = "cuda" if torch.cuda.is_available() else "cpu"

        self.supported_languages = {
            'en': 'English',
            'hi': 'Hindi',
            'ur': 'Urdu',
            'te': 'Telugu',
            'ta': 'Tamil',
            'bn': 'Bengali',
            'gu': 'Gujarati',
            'kn': 'Kannada',
            'ml': 'Malayalam',
            'mr': 'Marathi',
            'pa': 'Punjabi',
            'or': 'Odia',
            'as': 'Assamese',
            'es': 'Spanish',
            'fr': 'French',
            'de': 'German',
            'it': 'Italian',
            'pt': 'Portuguese',
            'ru': 'Russian',
            'ja': 'Japanese',
            'ko': 'Korean',
            'zh': 'Chinese',
            'ar': 'Arabic',
            'tr': 'Turkish',
            'th': 'Thai',
            'vi': 'Vietnamese'
        }

        # User profile storage
        self.user_profile = {
            'category': None,
            'preferences': {}
        }

    def configure_gemini(self, api_key):
        """Configure Gemini API with provided key"""
        try:
            genai.configure(api_key=api_key)
            self.gemini_model = genai.GenerativeModel('gemini-pro')
            return "SUCCESS: Gemini API configured successfully!"
        except Exception as e:
            return f"ERROR: Error configuring Gemini API: {str(e)}"

    def initialize_granite_model(self):
        """Initialize IBM Granite speech model"""
        try:
            model_name = "ibm-granite/granite-speech-3.3-2b"
            self.granite_processor = AutoProcessor.from_pretrained(model_name)
            self.granite_tokenizer = self.granite_processor.tokenizer
            self.granite_model = AutoModelForSpeechSeq2Seq.from_pretrained(
                model_name,
                device_map=self.device,
                torch_dtype=torch.bfloat16
            )
            return "SUCCESS: IBM Granite model loaded successfully!"
        except Exception as e:
            return f"ERROR: Error loading Granite model: {str(e)}"

    def speech_to_text_granite(self, audio_file):
        """Convert speech to text using IBM Granite model"""
        if not audio_file:
            return "", "Please upload an audio file"

        if not self.granite_model:
            init_result = self.initialize_granite_model()
            if "ERROR" in init_result:
                return "", init_result

        try:
            # Load and process audio
            wav, sr = torchaudio.load(audio_file, normalize=True)

            # Resample to 16kHz if needed
            if sr != 16000:
                resampler = torchaudio.transforms.Resample(sr, 16000)
                wav = resampler(wav)

            # Ensure mono audio
            if wav.shape[0] > 1:
                wav = torch.mean(wav, dim=0, keepdim=True)

            # Create prompts
            system_prompt = "Knowledge Cutoff Date: April 2024.\nToday's Date: April 9, 2025.\nYou are Granite, developed by IBM. You are a helpful AI assistant"
            user_prompt = "<|audio|>can you transcribe the speech into a written format?"

            chat = [
                dict(role="system", content=system_prompt),
                dict(role="user", content=user_prompt),
            ]

            prompt = self.granite_tokenizer.apply_chat_template(
                chat, tokenize=False, add_generation_prompt=True
            )

            # Process with model
            model_inputs = self.granite_processor(
                prompt, wav, device=self.device, return_tensors="pt"
            ).to(self.device)

            model_outputs = self.granite_model.generate(
                **model_inputs, max_new_tokens=200, do_sample=False, num_beams=1
            )

            # Extract transcription
            num_input_tokens = model_inputs["input_ids"].shape[-1]
            new_tokens = torch.unsqueeze(model_outputs[0, num_input_tokens:], dim=0)
            output_text = self.granite_tokenizer.batch_decode(
                new_tokens, add_special_tokens=False, skip_special_tokens=True
            )

            transcribed_text = output_text[0].strip()
            return transcribed_text, "SUCCESS: Speech transcribed successfully with IBM Granite!"

        except Exception as e:
            return "", f"ERROR: Speech-to-text error: {str(e)}"

    def translate_text(self, text, source_lang, target_lang, api_key):
        """Translate text using Gemini"""
        if not self.gemini_model:
            config_result = self.configure_gemini(api_key)
            if "ERROR" in config_result:
                return text, config_result

        try:
            source_name = self.supported_languages.get(source_lang, source_lang)
            target_name = self.supported_languages.get(target_lang, target_lang)

            prompt = f"Translate this text from {source_name} to {target_name}. Only return the translation, nothing else: {text}"

            response = self.gemini_model.generate_content(prompt)
            translated_text = response.text.strip()

            return translated_text, f"SUCCESS: Translated from {source_name} to {target_name}"
        except Exception as e:
            return text, f"ERROR: Translation failed: {str(e)}"

    def detect_language(self, text):
        """Detect language of input text"""
        try:
            detected_lang = detect(text)
            if detected_lang in self.supported_languages:
                return detected_lang, self.supported_languages[detected_lang]
            else:
                return 'en', 'English (Fallback)'
        except LangDetectException:
            return 'en', 'English (Default)'

    def enhance_text_with_gemini(self, text, user_category):
        """Use Gemini to enhance text based on user category"""
        if not self.gemini_model:
            return text, "Gemini not configured"

        try:
            category_prompts = {
                'student': f"Enhance this text to make it more educational and clear for a student: {text}",
                'working_professional': f"Make this text more professional and concise for a working professional: {text}",
                'visually_impaired': f"Make this text more descriptive and accessible for someone who is visually impaired, adding context where helpful: {text}",
                'other': f"Improve the clarity and flow of this text: {text}"
            }

            prompt = category_prompts.get(user_category, category_prompts['other'])
            response = self.gemini_model.generate_content(prompt)
            enhanced_text = response.text

            return enhanced_text, "SUCCESS: Text enhanced by Gemini"
        except Exception as e:
            return text, f"ERROR: Gemini enhancement failed: {str(e)}"

    def text_to_speech(self, text, lang_code, slow_speech=False):
        """Convert text to speech"""
        try:
            tts = gTTS(text=text, lang=lang_code, slow=slow_speech)
            audio_buffer = io.BytesIO()
            tts.write_to_fp(audio_buffer)
            audio_buffer.seek(0)

            with tempfile.NamedTemporaryFile(delete=False, suffix='.mp3') as tmp_file:
                tmp_file.write(audio_buffer.getvalue())
                tmp_file_path = tmp_file.name

            return tmp_file_path
        except Exception as e:
            raise Exception(f"TTS Error: {str(e)}")

    def extract_text_from_pdf(self, pdf_file):
        """Extract text from uploaded PDF file"""
        try:
            if pdf_file is None:
                return ""

            pdf_reader = PyPDF2.PdfReader(pdf_file)
            text = ""
            for page in pdf_reader.pages:
                text += page.extract_text() + "\n"
            return text.strip()
        except Exception as e:
            return f"Error extracting PDF text: {str(e)}"

    def process_complete_workflow(self, input_text, pdf_file, audio_file,
                                target_language, gemini_key, use_granite_stt,
                                user_category, enhance_with_gemini, slow_speech):
        """Complete workflow: PDF extraction, STT, translation, and TTS"""
        try:
            # Setup Gemini if key provided
            gemini_status = ""
            if gemini_key and gemini_key.strip():
                gemini_status = self.configure_gemini(gemini_key.strip())

            # Extract text from PDF if provided
            pdf_text = ""
            if pdf_file:
                pdf_text = self.extract_text_from_pdf(pdf_file)

            # Transcribe audio if provided and Granite STT is enabled
            audio_text = ""
            if audio_file and use_granite_stt:
                if self.granite_model is None:
                    granite_status = self.initialize_granite_model()
                audio_text, _ = self.speech_to_text_granite(audio_file)

            # Combine all text sources
            combined_text = []
            if input_text and input_text.strip():
                combined_text.append(f"Input Text:\n{input_text.strip()}")
            if pdf_text:
                combined_text.append(f"PDF Content:\n{pdf_text[:2000]}{'...' if len(pdf_text) > 2000 else ''}")
            if audio_text:
                combined_text.append(f"Audio Transcription:\n{audio_text}")

            if not combined_text:
                return "No text to process", "", None, None, gemini_status

            source_text = "\n\n".join(combined_text)

            # Detect language
            lang_code, lang_name = self.detect_language(source_text)
            detection_info = f"Detected Language: {lang_name} ({lang_code})"

            # Enhance text if requested
            processed_text = source_text
            enhancement_info = ""
            if enhance_with_gemini:
                processed_text, enhancement_status = self.enhance_text_with_gemini(source_text, user_category)
                enhancement_info = f"Gemini Enhancement: {enhancement_status}"

            # Translate text if needed
            translated_text = processed_text
            if target_language != "English":
                target_lang_name = self.supported_languages.get(target_language.lower(), target_language)
                translated_text, _ = self.translate_text(processed_text, lang_code, target_language.lower(), gemini_key)

            # Generate TTS for original text
            original_audio = self.text_to_speech(processed_text, lang_code, slow_speech)

            # Generate TTS for translated text
            translated_audio = None
            if target_language != "English" and translated_text != processed_text:
                target_code = target_language.lower()[:2]  # Get language code
                if target_code in self.supported_languages:
                    translated_audio = self.text_to_speech(translated_text, target_code, slow_speech)

            status_message = f"SUCCESS: Processing completed!\n{detection_info}\n{enhancement_info}"

            return (processed_text, translated_text, original_audio,
                   translated_audio, status_message)

        except Exception as e:
            return f"Error in workflow: {str(e)}", "", None, None, "Error occurred"

# Initialize the TTS system
tts_system = MultilingualTTSWithGranite()

# Enhanced CSS with black background and green accents
custom_css = """
/* Force dark theme with black background */
.gradio-container {
    font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif !important;
    background: #000000 !important;
    color: #ffffff !important;
}

body, .gradio-container, .app, .main, .block, .container {
    background: #000000 !important;
    color: #ffffff !important;
}

/* Input styling */
.gr-textbox, .gr-dropdown, .gr-file, .gr-checkbox, .gr-radio, textarea, input, select {
    background: #1a1a1a !important;
    color: #ffffff !important;
    border: 2px solid #00ff00 !important;
    border-radius: 8px !important;
}

/* Button styling */
.gr-button {
    background: linear-gradient(135deg, #00ff00 0%, #00cc00 100%) !important;
    color: #000000 !important;
    border: none !important;
    font-weight: bold !important;
    border-radius: 8px !important;
    padding: 10px 20px !important;
    transition: all 0.3s ease !important;
}

.gr-button:hover {
    background: linear-gradient(135deg, #00cc00 0%, #009900 100%) !important;
    transform: translateY(-2px) !important;
    box-shadow: 0 4px 15px rgba(0, 255, 0, 0.4) !important;
}

/* Headers and labels */
label, .gr-form label, .gr-block-title, h1, h2, h3, h4, h5, h6 {
    color: #ffffff !important;
    font-weight: bold !important;
}

/* Tab styling */
.gr-tab-nav button {
    background: #1a1a1a !important;
    color: #ffffff !important;
    border: 1px solid #00ff00 !important;
}

.gr-tab-nav button.selected {
    background: linear-gradient(135deg, #00ff00 0%, #00cc00 100%) !important;
    color: #000000 !important;
}

/* Output areas */
.gr-textbox[data-testid="textbox"] {
    background: #1a1a1a !important;
    color: #ffffff !important;
    border: 1px solid #00ff00 !important;
}

/* Audio player */
audio {
    background: #1a1a1a !important;
    border: 1px solid #00ff00 !important;
    border-radius: 8px !important;
}

/* File upload */
.gr-file-upload {
    background: #1a1a1a !important;
    border: 2px dashed #00ff00 !important;
    color: #ffffff !important;
    border-radius: 8px !important;
}

/* Special containers */
.title-header {
    text-align: center;
    background: linear-gradient(135deg, #1a1a1a 0%, #2a2a2a 100%);
    color: #00ff00;
    padding: 25px;
    border-radius: 15px;
    margin-bottom: 20px;
    border: 2px solid #00ff00;
}

.feature-section {
    background: #1a1a1a;
    color: #ffffff;
    padding: 20px;
    border-radius: 12px;
    margin: 15px 0;
    border-left: 5px solid #00ff00;
}

/* Accordion */
.gr-accordion {
    background: #1a1a1a !important;
    border: 1px solid #00ff00 !important;
    border-radius: 8px !important;
}

/* Progress bars */
.gr-progress {
    background: #00ff00 !important;
}

/* Links */
a {
    color: #00ff00 !important;
}
"""

def create_interface():
    with gr.Blocks(css=custom_css, title="AI-Powered Multilingual Speech System") as interface:

        # Header
        gr.HTML("""
        <div class="title-header">
            <h1>Echo Verse - AI Powered Audiobook Creation Tool</h1>
            <h3>Gemini AI + IBM Granite + Google TTS</h3>
            <p>Professional audiobook creation with Text-to-Speech and Speech-to-Text in 25+ languages!</p>
        </div>
        """)

        # User Category Selection
        gr.HTML('<h3 style="color: #00ff00;">Welcome! Please tell us about yourself:</h3>')

        user_category = gr.Radio(
            choices=["student", "working_professional", "visually_impaired", "other"],
            label="I am a:",
            value="student",
            interactive=True
        )

        # API Configuration
        with gr.Accordion("API Configuration", open=True):
            api_key_input = gr.Textbox(
                label="Gemini API Key",
                placeholder="Enter your Google Gemini API key here...",
                type="password",
                info="Get your free API key from: https://makersuite.google.com/app/apikey"
            )

            granite_init_btn = gr.Button("Initialize IBM Granite Model", variant="secondary")
            granite_status = gr.Textbox(label="Granite Model Status", interactive=False)

        # Main Features Tabs
        with gr.Tabs():

            # Complete Workflow Tab
            with gr.TabItem("Complete Workflow"):
                gr.HTML('<div class="feature-section">')
                gr.HTML("<h3>Multi-Source Processing</h3>")

                with gr.Row():
                    with gr.Column():
                        # Input sources
                        input_text = gr.Textbox(
                            label="Text Input",
                            placeholder="Type or paste text in any language",
                            lines=5
                        )

                        pdf_upload = gr.File(
                            label="Upload PDF Document",
                            file_types=[".pdf"],
                            type="filepath"
                        )

                        audio_upload = gr.File(
                            label="Upload Audio File",
                            file_types=[".wav", ".mp3", ".m4a", ".ogg"],
                            type="filepath"
                        )

                        # Options
                        target_language = gr.Dropdown(
                            label="Target Language",
                            choices=["English", "Spanish", "French", "German", "Hindi", "Chinese", "Japanese"],
                            value="English"
                        )

                        use_granite_stt = gr.Checkbox(
                            label="Use IBM Granite for Speech-to-Text",
                            value=True
                        )

                        enhance_with_gemini = gr.Checkbox(
                            label="Enhance text with Gemini AI",
                            value=True
                        )

                        slow_speech = gr.Checkbox(
                            label="Generate slow speech",
                            value=False
                        )

                        process_btn = gr.Button("Process All Sources", variant="primary", size="lg")

                # Output Section
                with gr.Row():
                    with gr.Column():
                        gr.HTML("<h3>Original Text & Audio</h3>")
                        original_output = gr.Textbox(
                            label="Processed Original Text",
                            lines=8,
                            interactive=False
                        )
                        original_audio = gr.Audio(label="Original Audio", type="filepath")

                    with gr.Column():
                        gr.HTML("<h3>Translated Text & Audio</h3>")
                        translated_output = gr.Textbox(
                            label="Translated Text",
                            lines=8,
                            interactive=False
                        )
                        translated_audio = gr.Audio(label="Translated Audio", type="filepath")

                status_output = gr.Textbox(
                    label="Processing Status",
                    lines=3,
                    interactive=False
                )

                gr.HTML('</div>')

            # Text-to-Speech Tab
            with gr.TabItem("Text-to-Speech Only"):
                with gr.Row():
                    with gr.Column():
                        tts_text_input = gr.Textbox(
                            label="Enter text to convert to speech",
                            placeholder="Type text in any language",
                            lines=5
                        )

                        tts_enhance_checkbox = gr.Checkbox(
                            label="Enhance text with Gemini AI",
                            value=True
                        )

                        tts_slow_speech_checkbox = gr.Checkbox(
                            label="Generate slow speech",
                            value=False
                        )

                        tts_generate_btn = gr.Button("Generate Speech", variant="primary")

                        tts_audio_output = gr.Audio(label="Generated Speech", type="filepath")
                        tts_status_output = gr.Textbox(label="Status", lines=3, interactive=False)
                        tts_enhanced_text_output = gr.Textbox(label="Enhanced Text", lines=4, interactive=False)

            # Speech-to-Text Tab
            with gr.TabItem("Speech-to-Text Only"):
                gr.HTML('<h3>IBM Granite Speech Recognition</h3>')

                stt_audio_input = gr.Audio(
                    label="Upload Audio File",
                    type="filepath"
                )

                stt_transcribe_btn = gr.Button("Transcribe with Granite", variant="secondary")

                stt_transcribed_output = gr.Textbox(
                    label="Transcribed Text",
                    lines=5,
                    interactive=False
                )

                stt_status = gr.Textbox(
                    label="Transcription Status",
                    lines=2,
                    interactive=False
                )

        # Quick Test Samples
        with gr.Accordion("Quick Test Samples", open=False):
            gr.HTML("<h4>Try these sample texts:</h4>")

            with gr.Row():
                sample_hindi = gr.Button("Hindi Sample")
                sample_telugu = gr.Button("Telugu Sample")
                sample_urdu = gr.Button("Urdu Sample")
                sample_english = gr.Button("English Sample")

        # Language Support Information
        with gr.Accordion("Supported Languages", open=False):
            lang_list = []
            for code, name in tts_system.supported_languages.items():
                lang_list.append(f"• {name} ({code})")

            gr.HTML(f"""
            <div style="background: #1a1a1a; padding: 15px; border-radius: 10px; border: 1px solid #00ff00;">
                <div style="columns: 3; column-gap: 20px; color: #ffffff;">
                    {chr(10).join(lang_list)}
                </div>
            </div>
            """)

        # Event Handlers

        # Complete Workflow
        process_btn.click(
            fn=tts_system.process_complete_workflow,
            inputs=[
                input_text, pdf_upload, audio_upload, target_language,
                api_key_input, use_granite_stt, user_category,
                enhance_with_gemini, slow_speech
            ],
            outputs=[
                original_output, translated_output,
                original_audio, translated_audio, status_output
            ]
        )

        # TTS Only
        def process_tts_only(text, api_key, category, enhance, slow):
            if not text.strip():
                return None, "Please enter some text", ""

            if not api_key.strip():
                return None, "Please enter your Gemini API key", ""

            # Configure Gemini
            config_result = tts_system.configure_gemini(api_key)
            if "ERROR" in config_result:
                return None, config_result, ""

            # Detect language
            lang_code, lang_name = tts_system.detect_language(text)

            processed_text = text
            enhancement_info = ""

            if enhance:
                processed_text, enhancement_status = tts_system.enhance_text_with_gemini(text, category)
                enhancement_info = f"Gemini Enhancement: {enhancement_status}"

            try:
                audio_path = tts_system.text_to_speech(processed_text, lang_code, slow)
                status_message = f"SUCCESS: Speech generated!\nDetected Language: {lang_name}\n{enhancement_info}"
                return audio_path, status_message, processed_text
            except Exception as e:
                return None, f"ERROR: {str(e)}", ""

        tts_generate_btn.click(
            fn=process_tts_only,
            inputs=[tts_text_input, api_key_input, user_category, tts_enhance_checkbox, tts_slow_speech_checkbox],
            outputs=[tts_audio_output, tts_status_output, tts_enhanced_text_output]
        )

        # Granite Model Initialization
        granite_init_btn.click(
            fn=tts_system.initialize_granite_model,
            outputs=granite_status
        )

        # Speech-to-Text
        stt_transcribe_btn.click(
            fn=tts_system.speech_to_text_granite,
            inputs=stt_audio_input,
            outputs=[stt_transcribed_output, stt_status]
        )

        # Sample text buttons
        sample_hindi.click(
            lambda: "नमस्ते! यह एक उन्नत बहुभाषी टेक्स्ट टू स्पीच सिस्टम है।",
            outputs=input_text
        )

        sample_telugu.click(
            lambda: "నమస్కారం! ఇది ఒక అధునాతన బహుభాషా టెక్స్ట్ టు స్పీచ్ సిస్టమ్.",
            outputs=input_text
        )

        sample_urdu.click(
            lambda: "السلام علیکم! یہ ایک جدید کثیر لسانی ٹیکسٹ ٹو اسپیچ سسٹم ہے۔",
            outputs=input_text
        )

        sample_english.click(
            lambda: "Hello! This is an advanced multilingual text-to-speech system powered by Gemini AI and IBM Granite.",
            outputs=input_text
        )

    return interface

# Installation function
def install_requirements():
    """Install required packages for Google Colab"""
    import subprocess
    import sys

    packages = [
        'gradio',
        'google-generativeai',
        'gtts',
        'langdetect',
        'transformers>=4.52.4',
        'torchaudio',
        'peft',
        'soundfile',
        'huggingface-hub',
        'PyPDF2'
    ]

    for package in packages:
        try:
            subprocess.check_call([sys.executable, '-m', 'pip', 'install', package])
            print(f"SUCCESS: {package} installed successfully")
        except subprocess.CalledProcessError:
            print(f"ERROR: Failed to install {package}")

def run_app():
    """Launch the Echo Verse audiobook creation tool"""
    print("Starting Echo Verse - AI-Powered Audiobook Creation Tool...")
    print("The app will be accessible via the generated Gradio link")
    print(f"Device: {tts_system.device}")

    interface = create_interface()

    # Launch with public sharing for Colab compatibility
    interface.launch(
        share=True,
        debug=True,
        server_name="0.0.0.0",
        inbrowser=True,
        show_error=True
    )

# Main execution
if __name__ == "__main__":
    print("Installing required packages...")
    install_requirements()

    print("\n" + "="*70)
    print("ECHO VERSE - AI-POWERED AUDIOBOOK CREATION TOOL")
    print("="*70)
    print("Features:")
    print("   * Automatic language detection (25+ languages)")
    print("   * Gemini AI text enhancement")
    print("   * IBM Granite speech-to-text")
    print("   * High-quality audiobook generation")
    print("   * User category customization")
    print("   * Black UI with green accents")
    print("="*70)
    print("Required: Gemini API key from https://makersuite.google.com/app/apikey")
    print("GPU recommended for Granite model performance")
    print("="*70)

    run_app()

Installing required packages...
SUCCESS: gradio installed successfully
SUCCESS: google-generativeai installed successfully
SUCCESS: gtts installed successfully
SUCCESS: langdetect installed successfully
SUCCESS: transformers>=4.52.4 installed successfully
SUCCESS: torchaudio installed successfully
SUCCESS: peft installed successfully
SUCCESS: soundfile installed successfully
SUCCESS: huggingface-hub installed successfully
SUCCESS: PyPDF2 installed successfully

ECHO VERSE - AI-POWERED AUDIOBOOK CREATION TOOL
Features:
   * Automatic language detection (25+ languages)
   * Gemini AI text enhancement
   * IBM Granite speech-to-text
   * High-quality audiobook generation
   * User category customization
   * Black UI with green accents
Required: Gemini API key from https://makersuite.google.com/app/apikey
GPU recommended for Granite model performance
Starting Echo Verse - AI-Powered Audiobook Creation Tool...
The app will be accessible via the generated Gradio link
Device: cuda
Colab note